Static word embeddings with Glove and Word2Vec

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import re,string,unicodedata
from keras.preprocessing import text, sequence
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
from string import punctuation
import keras
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Dropout,Bidirectional,GRU
import tensorflow as tf

In [2]:
df = pd.read_json("data/Sarcasm_Headlines_Dataset_v2.json", lines=True)
df.head()

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...


In [3]:
del df['article_link']
df.head()

,is_sarcastic,headline
0,1,thirtysomething scientists unveil doomsday clo...
1,0,dem rep. totally nails why congress is falling...
2,0,eat your veggies: 9 deliciously different recipes
3,1,inclement weather prevents liar from getting t...
4,1,mother comes pretty close to using word 'strea...


In [4]:
nltk.download('stopwords')
stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\wangj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)
# Removing URL's
def remove_between_square_brackets(text):
    return re.sub(r'http\S+', '', text)
#Removing the stopwords from text
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop:
            final_text.append(i.strip())
    return " ".join(final_text)
#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    text = remove_stopwords(text)
    return text
#Apply function on review column
df['headline']=df['headline'].apply(denoise_text)

<>:7: SyntaxWarning: invalid escape sequence '\['
<>:7: SyntaxWarning: invalid escape sequence '\['
C:\Users\wangj\AppData\Local\Temp\ipykernel_24500\1508946720.py:7: SyntaxWarning: invalid escape sequence '\['
  return re.sub('\[[^]]*\]', '', text)
C:\Users\wangj\AppData\Local\Temp\ipykernel_24500\1508946720.py:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


In [6]:
!pip install gensim

Defaulting to user installation because normal site-packages is not writeable


Word2Vec:

In [7]:
import gensim
words = []
for i in df.headline.values:
    words.append(i.split())
#Dimension of vectors we are generating
EMBEDDING_DIM = 200

#Creating Word Vectors by Word2Vec Method (takes time...)
w2v_model = gensim.models.Word2Vec(sentences = words , vector_size=EMBEDDING_DIM , window = 5 , min_count = 1)

In [8]:
tokenizer = text.Tokenizer(num_words=35000)
tokenizer.fit_on_texts(words)
tokenized_train = tokenizer.texts_to_sequences(words)
x = sequence.pad_sequences(tokenized_train, maxlen = 20)

In [9]:
vocab_size = len(tokenizer.word_index) + 1

In [19]:
# Function to create weight matrix from word2vec gensim model
def get_weight_matrix(model, vocab):
    # total vocabulary size plus 0 for unknown words
    vocab_size = len(vocab) + 1
    # define weight matrix dimensions with all 0
    weight_matrix = np.zeros((vocab_size, EMBEDDING_DIM))
    # step vocab, store vectors using the Tokenizer's integer mapping
    for word, i in vocab.items():
        if word in model.wv:
            weight_matrix[i] = model.wv[word]
    return weight_matrix

In [20]:
embedding_vectors = get_weight_matrix(w2v_model, tokenizer.word_index)

In [30]:
#Defining Neural Network
model = Sequential()
#Non-trainable embeddidng layer
model.add(Embedding(vocab_size, output_dim=EMBEDDING_DIM, weights=[embedding_vectors], input_length=20, trainable=True))
#LSTM 
model.add(Bidirectional(LSTM(units=128 , recurrent_dropout = 0.3 , dropout = 0.3,return_sequences = True)))
model.add(Bidirectional(GRU(units=32 , recurrent_dropout = 0.1 , dropout = 0.1)))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer=keras.optimizers.Adam(learning_rate = 0.01), loss='binary_crossentropy', metrics=[tf.keras.metrics.F1Score(dtype=tf.float32)], run_eagerly=True)

In [31]:
x_train, x_test, y_train, y_test = train_test_split(x, df.is_sarcastic.values.astype('float32') , test_size = 0.3 , random_state = 0)

In [ ]:
history = model.fit(x_train, y_train, batch_size = 128 , validation_data = (x_test,y_test) , epochs = 5)

Epoch 1/3


157/157 [==============================] - 265s 2s/step - loss: 0.5182 - f1_score: 0.6483 - val_loss: 0.4209 - val_f1_score: 0.6384
Epoch 2/3
157/157 [==============================] - 265s 2s/step - loss: 0.5182 - f1_score: 0.6483 - val_loss: 0.4209 - val_f1_score: 0.6384
Epoch 2/3
157/157 [==============================] - 262s 2s/step - loss: 0.1473 - f1_score: 0.6483 - val_loss: 0.5300 - val_f1_score: 0.6384
Epoch 3/3
157/157 [==============================] - 262s 2s/step - loss: 0.1473 - f1_score: 0.6483 - val_loss: 0.5300 - val_f1_score: 0.6384
Epoch 3/3
157/157 [==============================] - 262s 2s/step - loss: 0.0387 - f1_score: 0.6483 - val_loss: 0.7769 - val_f1_score: 0.6384


In [34]:
# Get predictions
y_pred_prob = model.predict(x_test)
y_pred = (y_pred_prob > 0.5).astype(int).flatten()

# Calculate metrics
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1-Score:  {f1:.4f}")

# Also print classification report for detailed metrics
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['Not Sarcastic', 'Sarcastic']))

269/269 [==============================] - 128s 478ms/step
Accuracy:  0.7879
Precision: 0.7662
Recall:    0.7881
F1-Score:  0.7770

Classification Report:
Accuracy:  0.7879
Precision: 0.7662
Recall:    0.7881
F1-Score:  0.7770

Classification Report:
               precision    recall  f1-score   support

Not Sarcastic       0.81      0.79      0.80      4560
    Sarcastic       0.77      0.79      0.78      4026

     accuracy                           0.79      8586
    macro avg       0.79      0.79      0.79      8586
 weighted avg       0.79      0.79      0.79      8586

               precision    recall  f1-score   support

Not Sarcastic       0.81      0.79      0.80      4560
    Sarcastic       0.77      0.79      0.78      4026

     accuracy                           0.79      8586
    macro avg       0.79      0.79      0.79      8586
 weighted avg       0.79      0.79      0.79      8586



Glove:

In [6]:
x_train,x_test,y_train,y_test = train_test_split(df.headline,df.is_sarcastic, test_size = 0.3 , random_state = 0)

In [7]:
max_features = 35000
maxlen = 200

In [8]:
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(x_train)
tokenized_train = tokenizer.texts_to_sequences(x_train)
x_train = sequence.pad_sequences(tokenized_train, maxlen=maxlen)

In [9]:
tokenized_test = tokenizer.texts_to_sequences(x_test)
X_test = sequence.pad_sequences(tokenized_test, maxlen=maxlen)

In [10]:
EMBEDDING_FILE = 'data/glove.twitter.27B.25d.txt'

In [11]:
def get_coefs(word, *arr): 
    return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(EMBEDDING_FILE))

In [19]:
all_embs = np.stack(list(embeddings_index.values()))
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = len(word_index) + 1  # +1 for padding token at index 0
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: 
        embedding_matrix[i] = embedding_vector

In [20]:
batch_size = 128
epochs = 2
embed_size = 25

In [21]:
model = Sequential()
#Non-trainable embeddidng layer
model.add(Embedding(nb_words, output_dim=embed_size, weights=[embedding_matrix], input_length=200, trainable=True))
#LSTM 
model.add(Bidirectional(LSTM(units=128 , recurrent_dropout = 0.5 , dropout = 0.5)))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer=keras.optimizers.Adam(learning_rate = 0.01), loss='binary_crossentropy', metrics=['acc'])

In [22]:
history = model.fit(x_train, y_train, batch_size = batch_size , validation_data = (X_test,y_test) , epochs = 2)

Epoch 1/2
157/157 [==============================] - 78s 469ms/step - loss: 0.5625 - acc: 0.7082 - val_loss: 0.4669 - val_acc: 0.7838
Epoch 2/2
157/157 [==============================] - 78s 469ms/step - loss: 0.5625 - acc: 0.7082 - val_loss: 0.4669 - val_acc: 0.7838
Epoch 2/2
157/157 [==============================] - 75s 475ms/step - loss: 0.3359 - acc: 0.8555 - val_loss: 0.3996 - val_acc: 0.8280


In [24]:
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int).flatten()

# Calculate metrics
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1-Score:  {f1:.4f}")

# Also print classification report for detailed metrics
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['Not Sarcastic', 'Sarcastic']))

269/269 [==============================] - 8s 30ms/step
Accuracy:  0.8280
Precision: 0.8719
Recall:    0.7422
F1-Score:  0.8018

Classification Report:
               precision    recall  f1-score   support

Not Sarcastic       0.80      0.90      0.85      4560
    Sarcastic       0.87      0.74      0.80      4026

     accuracy                           0.83      8586
    macro avg       0.84      0.82      0.82      8586
 weighted avg       0.83      0.83      0.83      8586

Accuracy:  0.8280
Precision: 0.8719
Recall:    0.7422
F1-Score:  0.8018

Classification Report:
               precision    recall  f1-score   support

Not Sarcastic       0.80      0.90      0.85      4560
    Sarcastic       0.87      0.74      0.80      4026

     accuracy                           0.83      8586
    macro avg       0.84      0.82      0.82      8586
 weighted avg       0.83      0.83      0.83      8586



LLM Finetune:

In [ ]:
!pip install transformers datasets accelerate

In [9]:
!pip install typing_extensions>=4.8.0

In [6]:
import torch

print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Device:", torch.cuda.get_device_name(0))


CUDA available: True
Device: NVIDIA GeForce RTX 4070 Laptop GPU


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset
import torch

# Choose model (you can change to 'roberta-base' if preferred)
model_name = "bert-base-uncased"

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

In [ ]:
# Prepare data - split the original dataframe
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['headline'].tolist(), 
    df['is_sarcastic'].tolist(), 
    test_size=0.3, 
    random_state=0
)

# The code below was generated by AI; see [1].
# Create datasets
train_dataset = Dataset.from_dict({'text': train_texts, 'label': train_labels})
test_dataset = Dataset.from_dict({'text': test_texts, 'label': test_labels})

In [9]:
# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128)

# Tokenize datasets
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/20033 [00:00<?, ? examples/s]

Map:   0%|          | 0/8586 [00:00<?, ? examples/s]

In [ ]:
# The code below was generated by AI; see [2].
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy='epoch',
    save_strategy='epoch',
    learning_rate=2e-5,
    fp16=True, 
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model='f1',
)

In [11]:
# Define compute metrics function
def compute_metrics(eval_pred):
    from sklearn.metrics import accuracy_score, precision_recall_fscore_support
    
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    acc = accuracy_score(labels, predictions)
    
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [12]:
# Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics,
)

# Fine-tune the model
trainer.train()

C:\Users\wangj\AppData\Roaming\Python\Python312\site-packages\accelerate\accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


  0%|          | 0/3135 [00:00<?, ?it/s]

{'loss': 0.5609, 'grad_norm': 6.999548435211182, 'learning_rate': 1.936842105263158e-05, 'epoch': 0.16}
{'loss': 0.4531, 'grad_norm': 6.616657733917236, 'learning_rate': 1.8730462519936205e-05, 'epoch': 0.32}
{'loss': 0.4531, 'grad_norm': 6.616657733917236, 'learning_rate': 1.8730462519936205e-05, 'epoch': 0.32}
{'loss': 0.3792, 'grad_norm': 6.039819240570068, 'learning_rate': 1.8092503987240832e-05, 'epoch': 0.48}
{'loss': 0.3792, 'grad_norm': 6.039819240570068, 'learning_rate': 1.8092503987240832e-05, 'epoch': 0.48}
{'loss': 0.3654, 'grad_norm': 5.852889537811279, 'learning_rate': 1.7454545454545456e-05, 'epoch': 0.64}
{'loss': 0.3654, 'grad_norm': 5.852889537811279, 'learning_rate': 1.7454545454545456e-05, 'epoch': 0.64}
{'loss': 0.3591, 'grad_norm': 7.357544422149658, 'learning_rate': 1.6816586921850083e-05, 'epoch': 0.8}
{'loss': 0.3591, 'grad_norm': 7.357544422149658, 'learning_rate': 1.6816586921850083e-05, 'epoch': 0.8}
{'loss': 0.3133, 'grad_norm': 4.97999382019043, 'learning_

  0%|          | 0/269 [00:00<?, ?it/s]

{'eval_loss': 0.3031638264656067, 'eval_accuracy': 0.8725832750989984, 'eval_f1': 0.8706550011823126, 'eval_precision': 0.8307761732851986, 'eval_recall': 0.914555389965226, 'eval_runtime': 10.476, 'eval_samples_per_second': 819.589, 'eval_steps_per_second': 25.678, 'epoch': 1.0}
{'loss': 0.2559, 'grad_norm': 10.363936424255371, 'learning_rate': 1.5540669856459333e-05, 'epoch': 1.12}
{'loss': 0.2559, 'grad_norm': 10.363936424255371, 'learning_rate': 1.5540669856459333e-05, 'epoch': 1.12}
{'loss': 0.2281, 'grad_norm': 10.028306007385254, 'learning_rate': 1.4902711323763957e-05, 'epoch': 1.28}
{'loss': 0.2281, 'grad_norm': 10.028306007385254, 'learning_rate': 1.4902711323763957e-05, 'epoch': 1.28}
{'loss': 0.2391, 'grad_norm': 11.0701265335083, 'learning_rate': 1.426475279106858e-05, 'epoch': 1.44}
{'loss': 0.2391, 'grad_norm': 11.0701265335083, 'learning_rate': 1.426475279106858e-05, 'epoch': 1.44}
{'loss': 0.2127, 'grad_norm': 5.7563958168029785, 'learning_rate': 1.3626794258373206e-05

...

  0%|          | 0/269 [00:00<?, ?it/s]

{'eval_loss': 0.4996281862258911, 'eval_accuracy': 0.8895877009084556, 'eval_f1': 0.8790199081163859, 'eval_precision': 0.9039370078740158, 'eval_recall': 0.8554396423248882, 'eval_runtime': 9.7773, 'eval_samples_per_second': 878.157, 'eval_steps_per_second': 27.513, 'epoch': 4.0}
{'loss': 0.0267, 'grad_norm': 12.466241836547852, 'learning_rate': 3.425837320574163e-06, 'epoch': 4.15}
{'loss': 0.0267, 'grad_norm': 12.466241836547852, 'learning_rate': 3.425837320574163e-06, 'epoch': 4.15}
{'loss': 0.0422, 'grad_norm': 1.8953601121902466, 'learning_rate': 2.7878787878787885e-06, 'epoch': 4.31}
{'loss': 0.0422, 'grad_norm': 1.8953601121902466, 'learning_rate': 2.7878787878787885e-06, 'epoch': 4.31}
{'loss': 0.0337, 'grad_norm': 0.033049944788217545, 'learning_rate': 2.1499202551834134e-06, 'epoch': 4.47}
{'loss': 0.0337, 'grad_norm': 0.033049944788217545, 'learning_rate': 2.1499202551834134e-06, 'epoch': 4.47}
{'loss': 0.0293, 'grad_norm': 0.17403681576251984, 'learning_rate': 1.5119617224

  0%|          | 0/269 [00:00<?, ?it/s]

{'eval_loss': 0.5370590090751648, 'eval_accuracy': 0.8916841369671559, 'eval_f1': 0.8830188679245283, 'eval_precision': 0.8944954128440367, 'eval_recall': 0.8718330849478391, 'eval_runtime': 9.7865, 'eval_samples_per_second': 877.327, 'eval_steps_per_second': 27.487, 'epoch': 5.0}
{'train_runtime': 544.6081, 'train_samples_per_second': 183.921, 'train_steps_per_second': 5.756, 'train_loss': 0.16601591653991163, 'epoch': 5.0}
{'train_runtime': 544.6081, 'train_samples_per_second': 183.921, 'train_steps_per_second': 5.756, 'train_loss': 0.16601591653991163, 'epoch': 5.0}


TrainOutput(global_step=3135, training_loss=0.16601591653991163, metrics={'train_runtime': 544.6081, 'train_samples_per_second': 183.921, 'train_steps_per_second': 5.756, 'total_flos': 6588629715033600.0, 'train_loss': 0.16601591653991163, 'epoch': 5.0})

In [13]:
# Evaluate the model
eval_results = trainer.evaluate()

print("Evaluation Results:")
print(f"Accuracy:  {eval_results['eval_accuracy']:.4f}")
print(f"Precision: {eval_results['eval_precision']:.4f}")
print(f"Recall:    {eval_results['eval_recall']:.4f}")
print(f"F1-Score:  {eval_results['eval_f1']:.4f}")

  0%|          | 0/269 [00:00<?, ?it/s]

Evaluation Results:
Accuracy:  0.8918
Precision: 0.8813
Recall:    0.8890
F1-Score:  0.8851
